In [73]:
'''
yaml_selector.py

Utility to load a YAML tree and retrieve immediate children of a specified parent node.
'''
import numpy as np
import yaml
import os
from typing import Any, List, Optional

# A YAMLTree can be any Python structure returned by yaml.safe_load
YAMLTree = Any


def load_yaml(path: str, base_dir: str = '.') -> YAMLTree:
    """Load a YAML file from a relative or absolute path."""
    full_path = path if os.path.isabs(path) else os.path.join(base_dir, path)
    with open(full_path, 'r') as f:
        return yaml.safe_load(f)


def get_children(
    tree: YAMLTree,
    parent_path: str,
    base_path: str = ''
) -> Optional[List[str]]:
    """
    Return the list of immediate children keys or indices for the node at `parent_path`.

    - `tree` is the YAML structure.
    - `parent_path` is the dot/bracket-delimited path of the parent node (e.g. 'resource.aws_route_table').
    - `base_path` is used internally for recursion and should be left as default when calling.

    Returns a list of child labels (dict keys or list indices), or None if `parent_path` not found.
    """
    # Check if the current node is the target parent
    if base_path == parent_path or (base_path == '' and parent_path in ('<root>', '', 'root')):
        if isinstance(tree, dict):
            return list(tree.keys())
        if isinstance(tree, list):
            return [f"[{i}]" for i in range(len(tree))]
        return []  # Scalar node has no children

    # Recurse into dict
    if isinstance(tree, dict):
        for key, subtree in tree.items():
            child_path = f"{base_path}.{key}" if base_path else key
            result = get_children(subtree, parent_path, child_path)
            if result is not None:
                [print(item) for item in result] if isinstance(result, list) else None

                return result

    # Recurse into list
    elif isinstance(tree, list):
        for idx, item in enumerate(tree):
            child_path = f"{base_path}[{idx}]" if base_path else f"[{idx}]"
            result = get_children(item, parent_path, child_path)
            if result is not None:
                [print(item) for item in result] if isinstance(result, list) else None
                return result


    return None  # No match found

# Example Usage:
# --------------
# tree = load_yaml('resource.yaml', base_dir='relative/path')
# children = get_children(tree, 'resource.aws_route_table')
# print(children)  # e.g. ['private_kube']


In [74]:

# Example Usage
# -------------
# tree = load_yaml('resource.yaml', base_dir='relative/path')\#
# rels = get_parent_child_relationships(tree)
# for parent, children in rels:
#     print(f"Parent: {parent}\n  Children: {children}\n")

tree = load_yaml('yaml_combined/resource.yaml')
children = get_children(tree, 'resource.aws_iam_role')

print("children:", children)  # e.g. ['private_kube']



[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
children: ['[0]', '[1]', '[2]', '[3]', '[4]', '[5]', '[6]', '[7]', '[8]', '[9]', '[10]']


In [82]:
#!/usr/bin/env python3
import os
import yaml

# 1) Custom loader to collect duplicates into lists
class MultiValueLoader(yaml.SafeLoader):
    pass

def construct_mapping(loader, node):
    mapping = {}
    for key_node, value_node in node.value:
        key = loader.construct_object(key_node, deep=True)
        val = loader.construct_object(value_node, deep=True)
        if key in mapping:
            if isinstance(mapping[key], list):
                mapping[key].append(val)
            else:
                mapping[key] = [mapping[key], val]
        else:
            mapping[key] = val
    return mapping

MultiValueLoader.add_constructor(
    yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG,
    construct_mapping
)

# 2) Merge logic: union differing values into sequences

def merge_values(a, b):
    if isinstance(a, dict) and isinstance(b, dict):
        return merge_dicts(a, b)
    if isinstance(a, list) or isinstance(b, list):
        a_list = a if isinstance(a, list) else [a]
        b_list = b if isinstance(b, list) else [b]
        merged = []
        for item in a_list + b_list:
            if item not in merged:
                merged.append(item)
        return merged
    if a == b:
        return a
    return [a, b]


import copy

def merge_dicts(target, incoming):
    for key, val in incoming.items():
        if key in target:
            # existing key ⇒ recurse/merge
            target[key] = merge_values(target[key], val)
        else:
            # new key ⇒ ensure we get a fresh copy (no aliasing)
            if isinstance(val, dict):
                # either deep‐copy or merge into a new dict
                target[key] = merge_dicts({}, val)
            elif isinstance(val, list):
                target[key] = copy.deepcopy(val)
            else:
                target[key] = val
    return target


# 3) Recursively collect and merge all YAML in a directory

def collect_and_merge_yaml(root_dir):
    merged = {}
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.endswith(('.yaml', '.yml')):
                path = os.path.join(dirpath, fname)
                try:
                    with open(path, 'r') as f:
                        docs = list(yaml.load_all(f, Loader=MultiValueLoader))
                    for doc in docs:
                        if isinstance(doc, dict):
                            merge_dicts(merged, doc)
                except Exception as e:
                    print(f"Warning: could not parse {path}: {e}")
    return merged

# 4) Custom dumper that enforces block-style lists indented under their parent
class IndentDumper(yaml.SafeDumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)

# Ensure block style for lists

def represent_block_list(dumper, data):
    return dumper.represent_sequence('tag:yaml.org,2002:seq', data, flow_style=False)

IndentDumper.add_representer(list, represent_block_list)

# 5) Run merge and dump

def main():
    root_dir = 'yaml_dir'  # directory containing your YAML files
    out_path = os.path.abspath('yaml_combined/resource.yaml')

    merged_tree = collect_and_merge_yaml(root_dir)

    yaml_text = yaml.dump(
        merged_tree,
        Dumper=IndentDumper,
        sort_keys=False,
        default_flow_style=False,
        indent=2,
    )

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, 'w') as outf:
        outf.write(yaml_text)

    print(f"Merged YAML written to {out_path}")

if __name__ == '__main__':
    main()


Merged YAML written to c:\Users\boadeyem\OneDrive - Indiana University\Documents\Masters-Career Documents\Portfolio\Projects\Gen3\yaml_combined\resource.yaml


In [ ]:
import os
base_dir = "yaml_combined/common/charts"
values_overrides = {
}
template_overrides = {
}
descriptions = {
    "vpc-controller": "VPC + Subnet + IGW + RouteTables via ACK",
    "iam-controller": "IAM Roles, Policies, Attachments via ACK",
    # add any other custom descriptions here...
}
controllers = [
    "vpc-controller",
    "iam-controller",
    "rds-controller",
    "s3-controller",
    "kms-controller",
    "ec2-controller",
    "autoscaling-controller",
    "sns-controller",
    "sqs-controller",
    "cwlogs-controller",
    "route53-controller",
    "peering-controller",
    "secretsmgr-controller",
    "cloudtrail-controller",
    "elasticsearch-controller",
    "wafv2-controller"
]
# helper function
def add_value_override(overrides: dict, controller: str, stub: str):
    """
    Add or overwrite the stub fo
    r `controller` in the overrides dict.
    """
    action = "Overriding" if controller in overrides else "Adding"
    print(f"{action} stub for '{controller}'")
    overrides[controller] = stub

def add_template_stub(templates: dict, controller: str, filename: str, stub: str):
    """
    Add or overwrite the stub for `filename` under `controller` in the templates dict.
    """
    if controller not in templates:
        templates[controller] = {}
    action = "Overriding" if filename in templates[controller] else "Adding"
    print(f"{action} stub for '{controller}/{filename}'")
    templates[controller][filename] = stub

    

In [ ]:
# example: add an RDS stub
rds_stub = """
enabled: true

dbInstance:
  identifier: "{{ .Release.Name }}-db"
  engine: postgres
  instanceClass: db.t3.micro
  allocatedStorage: 20
"""
add_value_override(values_overrides, "rds-controller", rds_stub)
vpc_stub =    """
apiVersion: v1
kind: SecurityGroup
metadata:
  name: {{ .Release.Name }}-sg
spec:
  description: "Allow SSH"
  rules:
    - protocol: tcp
      port: 22
      cidr: "0.0.0.0/0"
"""
# suppose template_files is your dict of controller→{ filename: content }
add_template_stub(template_overrides, "vpc-controller", "securitygroup.yaml", vpc_stub)

In [ ]:
# 3) Generate Chart.yaml content for each controller
chart_overrides = {
    name: f"""apiVersion: v2
name: {name}
description: {descriptions.get(name, name.replace('-', ' ').title() + " via ACK")}
type: application
version: 0.1.0
appVersion: "latest"
"""
    for name in controllers
}


for ctrl in controllers:
    if ctrl not in values_overrides:
        default_stub = f"""
# default values for {ctrl}
enabled: false
# TODO: fill in {ctrl}-specific settings
"""
        add_value_override(values_overrides, ctrl, default_stub)

#!/usr/bin/env python3
from pathlib import Path
import os

def bootstrap_subcharts(names, base):
    """
    Ensure each subchart dir and its templates/ folder exist.
    """
    for name in names:
        chart_dir     = os.path.join(base, name)
        templates_dir = os.path.join(chart_dir, "templates")
        os.makedirs(templates_dir, exist_ok=True)
    print("✅ Subchart directories + templates/ bootstrapped.")

def populate_subcharts(chart_defs, values_defs, template_files, names, base):
    """
    Write out Chart.yaml, values.yaml, and any number of templates for each subchart.

    - chart_defs:    { controller_name: chart_yaml_str, ... }
    - values_defs:   { controller_name: values_yaml_str, ... }
    - template_files:{ controller_name: { filename1: content1, filename2: content2, ... }, ... }
    - names:         [ controller_name, ... ]
    - base:          path to directory under which each <name>/templates lives
    """
    for name in names:
        chart_dir = Path(base) / name
        if not chart_dir.is_dir():
            print(f"⚠️  Skipping '{name}': directory not found.")
            continue

        # ─── Chart.yaml ──────────────────────────────────────────────────────────
        if name in chart_defs:
            (chart_dir / "Chart.yaml").write_text(chart_defs[name].lstrip() + "\n", encoding="utf-8")
            print(f"✏️  Wrote Chart.yaml for {name}")

        # ─── values.yaml ─────────────────────────────────────────────────────────
        if name in values_defs:
            (chart_dir / "values.yaml").write_text(values_defs[name].lstrip() + "\n", encoding="utf-8")
            print(f"✏️  Wrote values.yaml for {name}")

        # ─── arbitrary template files ───────────────────────────────────────────
        if name in template_files:
            tpl_dir = chart_dir / "templates"
            tpl_dir.mkdir(exist_ok=True)
            for filename, raw in template_files[name].items():
                path = tpl_dir / filename
                path.write_text(raw.lstrip() + "\n", encoding="utf-8")
                print(f"✏️  Wrote templates/{filename} for {name}")

# 1) Bootstrap folder structure
bootstrap_subcharts(controllers, base_dir)
# 4) Populate each subchart with its Chart.yaml and values.yaml
populate_subcharts(chart_overrides, values_overrides, template_overrides, controllers, base_dir)


In [7]:
#!/usr/bin/env python3
import os
from pathlib import Path
import textwrap
from string import Template

# Directory for per-controller scripts
SCRIPTS_DIR = Path("scripts") / "build"

# List of controllers
CONTROLLERS = [
    "s3-controller",
    "kms-controller",
    "ec2-controller",
    "autoscaling-controller",
    "sns-controller",
    "sqs-controller",
    "cwlogs-controller",
    "route53-controller",
    "peering-controller",
    "secretsmgr-controller",
    "cloudtrail-controller",
    "elasticsearch-controller",
    "wafv2-controller",
]
#remove -controller from the list 
# Common header to prepend to each generated script
HEADER = Template(textwrap.dedent('''\
#!/usr/bin/env python3
from string import Template

# Descriptions for controllers
controller = "${ctrl}-controller"
description = "
add_description_override(full_descriptions, controller, description)

# insert controller name into chart stub
${ctrl}_chart_stub1 = Template("""apiVersion: v2
name: $controller
description: $descriptions
version: 0.1.0
appVersion: "latest"
""")
${ctrl}_chart_stub1 = ${ctrl}_chart_stub1.substitute(controller=controller, descriptions=description)
add_value_override(chart_overrides, controller, ${ctrl}_chart_stub1)

${ctrl}_values_stub1 = """
"""
add_value_override(values_overrides, controller, ${ctrl}_values_stub1)

${ctrl}_template_stub1_1 = """
"""
add_template_stub(template_overrides, controller, "

${ctrl}_template_stub1_2 = """
"""
add_template_stub(template_overrides, controller, "

${ctrl}_template_stub1_3 = """
"""
add_template_stub(template_overrides, controller, "
'''))

def initialize_scripts(controllers, header, target_dir=SCRIPTS_DIR):
    """
    Create or overwrite one script per controller, each with the given header.
    """
    target_path = Path(target_dir)
    target_path.mkdir(parents=True, exist_ok=True)
    for ctrl in controllers:
        file_path = target_path / f"{ctrl}.py"
        base = ctrl.replace("-controller", "")
        print(base)
        rendered = header.safe_substitute(ctrl=base)
        file_path.write_text(rendered + "\n", encoding="utf-8")
        print(f"✏️  Initialized {file_path}")


def main():
    initialize_scripts(CONTROLLERS, HEADER)


if __name__ == "__main__":
    main()


s3
✏️  Initialized scripts\build\s3-controller.py
kms
✏️  Initialized scripts\build\kms-controller.py
ec2
✏️  Initialized scripts\build\ec2-controller.py
autoscaling
✏️  Initialized scripts\build\autoscaling-controller.py
sns
✏️  Initialized scripts\build\sns-controller.py
sqs
✏️  Initialized scripts\build\sqs-controller.py
cwlogs
✏️  Initialized scripts\build\cwlogs-controller.py
route53
✏️  Initialized scripts\build\route53-controller.py
peering
✏️  Initialized scripts\build\peering-controller.py
secretsmgr
✏️  Initialized scripts\build\secretsmgr-controller.py
cloudtrail
✏️  Initialized scripts\build\cloudtrail-controller.py
elasticsearch
✏️  Initialized scripts\build\elasticsearch-controller.py
wafv2
✏️  Initialized scripts\build\wafv2-controller.py
